This is https://www.tensorflow.org/tutorials/quickstart/beginner, initially forked from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb, and then extended - to self learn.

## Import TensorFlow

In [14]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import pandas as pd

TensorFlow version: 2.8.0


## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("Shapes: x_train", x_train.shape, "; y_train", y_train.shape, "; x_test", x_test.shape, "; ", y_test.shape, "\n")
print("Description of y_train:", pd.DataFrame(y_train).describe(), "\n")
print("y_train[:1] = ", y_train[:1])

Shapes: x_train (60000, 28, 28) ; y_train (60000,) ; x_test (10000, 28, 28) ;  (10000,) 

Description of y_train:                   0
count  60000.000000
mean       4.453933
std        2.889270
min        0.000000
25%        2.000000
50%        4.000000
75%        7.000000
max        9.000000 

y_train[:1] =  [5]


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [16]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [17]:
predictions = model(x_train[:1]).numpy()
print(predictions)

[[-0.67643166  0.7261473  -0.02020825 -0.09915751  0.29011926 -0.28185853
   0.14484969  0.45289686 -0.11767417 -1.115447  ]]


The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [18]:
tf.nn.softmax(predictions).numpy()

array([[0.04843291, 0.1969123 , 0.0933544 , 0.08626756, 0.12732342,
        0.07186236, 0.1101079 , 0.149831  , 0.08468487, 0.03122328]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [19]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [20]:
loss_fn(y_train[:1], predictions).numpy()

2.6330028

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [21]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [22]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2949 - accuracy: 0.9132
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1406 - accuracy: 0.9581
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1049 - accuracy: 0.9678
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0864 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0733 - accuracy: 0.9766


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [23]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0780 - accuracy: 0.9762 - 571ms/epoch - 2ms/step


[0.07804422080516815, 0.9761999845504761]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [24]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [25]:
# TODO range = :1 # :3
input = x_test[:1] # TODO range
probabilities = probability_model(input).numpy()
print(probabilities)

[[2.8533602e-08 1.6426807e-08 1.6673229e-06 1.7092911e-03 6.5192615e-11
  3.1124358e-08 1.4276659e-14 9.9828428e-01 2.8498466e-08 4.7811732e-06]]


In [26]:
test = y_test[:1] # TODO range
print(test)

[7]
